# Image Classification using Transfer Learning

This notebook demonstrates how to use a pre-trained convolutional neural network (CNN) for a new image classification task. This technique, known as **transfer learning**, is highly effective and allows us to achieve high accuracy with a relatively small dataset and less training time.

**Use Case:** Classifying images of cats and dogs.

**Model:** We will use the **MobileNetV2** model, pre-trained on the large-scale ImageNet dataset, and fine-tune it for our specific task.

## 1. Setup and Imports

First, we import the necessary libraries. Ensure you have installed all the packages from `requirements.txt`.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam

## 2. Data Preparation

This notebook uses the [Kaggle "Dogs vs. Cats"](https://www.kaggle.com/c/dogs-vs-cats/data) dataset.

**IMPORTANT:**
1. Download the `train.zip` file from the link above.
2. Create a `data` directory in the same folder as this notebook.
3. Unzip the contents and organize them into the following structure:
```
data/
└── train/
    ├── cat/  (contains all cat images, e.g., cat.0.jpg, cat.1.jpg)
    └── dog/  (contains all dog images, e.g., dog.0.jpg, dog.1.jpg)
```

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = (160, 160)
DATA_DIR = 'data/train/'

# Create TensorFlow datasets from the image directories
# We'll use 80% of the data for training and 20% for validation.
train_dataset = image_dataset_from_directory(DATA_DIR,
                                             shuffle=True,
                                             validation_split=0.2,
                                             subset='training',
                                             seed=42,
                                             image_size=IMG_SIZE,
                                             batch_size=BATCH_SIZE)

validation_dataset = image_dataset_from_directory(DATA_DIR,
                                                  shuffle=True,
                                                  validation_split=0.2,
                                                  subset='validation',
                                                  seed=42,
                                                  image_size=IMG_SIZE,
                                                  batch_size=BATCH_SIZE)

class_names = train_dataset.class_names
print(f"Class Names: {class_names}")

### Visualize a few images

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

### Configure dataset for performance
We use buffered prefetching to load images from disk without causing I/O blocking.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)

## 3. Build the Model using Transfer Learning

We will load the **MobileNetV2** model, pre-trained on ImageNet. We chop off the original classification layer (`include_top=False`) and replace it with our own.

In [ ]:
IMG_SHAPE = IMG_SIZE + (3,)

# Load the base model with weights pre-trained on ImageNet.
base_model = MobileNetV2(input_shape=IMG_SHAPE,
                           include_top=False,
                           weights='imagenet')

# Freeze the convolutional base to prevent its weights from being updated during training.
base_model.trainable = False

# Add a new classification head
inputs = Input(shape=IMG_SHAPE)
x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs) # Preprocess the input
x = base_model(x, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(1, activation='sigmoid')(x) # Sigmoid for binary classification

model = Model(inputs, outputs)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

## 4. Initial Training

We train the model for a few epochs with the base model frozen. This will only train the new `Dense` layers we added.

In [ ]:
initial_epochs = 5

history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)

### Plot learning curves

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## 5. Fine-Tuning

The initial training shows good results, but we can try to improve further by **fine-tuning**. This involves un-freezing the top layers of the base model and training the entire model with a very low learning rate.

In [ ]:
# Un-freeze the base model
base_model.trainable = True

# Let's see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

# Re-compile the model with a very low learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

### Continue training

In [ ]:
fine_tune_epochs = 5
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=validation_dataset)

## 6. Evaluation

Let's look at the learning curves for the entire process, including fine-tuning.

In [ ]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']
loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## 7. Conclusion

This notebook demonstrates the power of transfer learning. By using a pre-trained model, we were able to achieve high accuracy on our cats vs. dogs classification task with minimal training time and data. The process of initial training followed by fine-tuning is a standard and effective workflow for many computer vision problems.